# Type conversions in Rust

Rust type conversions fall into three categories:

* *Manual*
  * User-defined type conversions provided by implementing the From and Into traits

* *Semi-automatic*
  * Explicit casts between values using the as keyword

* *Automatic*
  * Implicit coercion into a new type

In [2]:
let x: u32 = 2;
let y: u64 = x;

Error: mismatched types

## User defined conversion

Thera are four traits for user-defined conversions:

* `From<T>`: defines a conversion from type `T` to `Self`
* `Into<T>`: defines a conversion from `Self` to type `T`
* `TryFrom<T>`: defines a fallible conversion from type `T` to `Self` - returns a `Result<Self, E>`
* `TryInto<T>`: defines a fallible conversion from `Self` to type `T` - returns a `Result<T, E>`

### Infallible conversions

* The type conversion traits `From` and `Into` are used for infallible conversions between types. 
* If you implement `From<T>` for type `U`, you get the implementation of `Into<U>` for type `T` for free. Rust uses blanket trait implementations to achieve this.

In [ ]:
mod explain {
    impl<T, U> Into<U> for T
    where
        U: From<T>,
    {
        fn into(self) -> U {
            U::from(self)
        }
    }
}

In [8]:
#[derive(Debug, Copy, Clone)]
pub struct ASNumber(pub u32);

pub fn is_private_asn(asn: ASNumber) -> bool {
    matches!(asn.0, 64512..=65534 | 4200000000..=4294967294)
}

In [9]:
is_private_asn(665)

Error: mismatched types

In [10]:
impl From<u32> for ASNumber {
    fn from(value: u32) -> Self {
        ASNumber(value)
    }
}

* Now it possible to convert from `u32` to `ASNumber` using the `into` method.

In [12]:
is_private_asn(665.into());

* Generic functions can be defined to accept any type that can be converted into a target type using the `Into` trait.

In [13]:
pub fn is_private_asn<T>(asn: T) -> bool
where
    T: Into<ASNumber>,
{
    let asn: ASNumber = asn.into();
    matches!(asn.0, 64512..=65534 | 4200000000..=4294967294)
}

In [15]:
println!("is_private_asn({}): {}", 665, is_private_asn(665));

is_private_asn(665): false


### Fallible conversions

* The type conversion traits `TryFrom` and `TryInto` are used for fallible conversions between types.

In [2]:
#[derive(Debug, Clone)]
pub struct Identifier(pub String); 

impl Identifier {
    pub fn is_valid(&self) -> bool {
        self.0.len() >= 4 && self.0.starts_with("ID-") && self.0.chars().skip(3).all(|c| c.is_ascii_digit())
    }
}

assert!(Identifier("ID-1234".to_string()).is_valid());
assert!(Identifier("ID-12".to_string()).is_valid());   
assert!(!Identifier("ID-AA".to_string()).is_valid());   
assert!(!Identifier("ID-12A".to_string()).is_valid());   

In [3]:
impl TryFrom<String> for Identifier {
    type Error = &'static str;

    fn try_from(value: String) -> Result<Self, Self::Error> {
        let identifier = Identifier(value);
        if identifier.is_valid() {
            Ok(identifier)
        } else {
            Err("Invalid Identifier format")
        }
    }
}

In [4]:
let id: Result<Identifier, _> = "ID-1234".to_string().try_into();
println!("{:?}", id);

Ok(Identifier("ID-1234"))


In [5]:
let id: Result<Identifier, _> = "ID-123!4".to_string().try_into();
println!("{:?}", id);

Err("Invalid Identifier format")


## Casts

* Explicit casts between types can be performed using the `as` keyword.

In [8]:
let x: u32 = 665;
let y: u64 = x as u64;

* Rust allows lossy conversions when casting between primitive types using `as`. For example, casting a `u64` to a `u32` will truncate the higher bits if the value exceeds the maximum value of `u32`.

In [9]:
let z: u8 = x as u8; // This will truncate the value if x > 255
println!("x: {}, y: {}, z: {}", x, y, z);

x: 665, y: 665, z: 153


* Lossy conversion is not allowed with the `From` and `Into` traits to ensure type safety at compile time.

In [ ]:
z = x.into(); // This will cause a compile-time error due to potential truncation

Error: the trait bound `u8: From<u32>` is not satisfied

* For consistency and safety, you should prefer `from/into` conversions over `as` casts, unless you understand and need the precise casting semantics provided by `as`.

## Coercion

* Coercion is an automatic conversion that the Rust compiler applies in certain situations:
    * Converting mutable references to immutable references (e.g., `&mut T` to `&T`)
    * Converting an array to a slice (e.g., `&[T; N]` to `&[T]`)
    * An item lifetime to a shorter lifetime (e.g., `&'a T` to `&'b T` where `'b: 'a`)

* There are two coercions whose behavior can be affected by user-defined types:
    * Deref coercion: When a type implements the `Deref` or `DerefMut` traits, Rust can automatically dereference it to access the underlying type. These coercions are commonly used with smart pointers like `Box`, `Rc`, and `Arc`.
    * Conversion to a trait object: When a type implements a trait, Rust can automatically convert references to that type into references to the corresponding trait object (e.g., `&T` to `&dyn Trait`).